In [1]:
%load_ext autoreload
%autoreload 2

In [92]:
import random
import time
from utils import *
from models import *
from attention import *

import torch
import numpy as np
import torch.optim as optim
from torch.utils import data
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Download the file
get_file()
path_to_file = 'spa-eng/spa.txt'

======== THE DATASET IS READY ========


In [4]:
# Try experimenting with the size of that dataset
num_examples = 118960
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

The number of line in the dataset is 118964


In [5]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.1)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

107064 107064 11896 11896


In [6]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Input Language; index to word mapping
2 ----> <start>
10 ----> tom
588 ----> piensa
5 ----> que
32 ----> mary
73 ----> puede
96 ----> hablar
119 ----> frances
4 ----> .
3 ----> <end>

Target Language; index to word mapping
2 ----> <start>
9 ----> tom
586 ----> thinks
31 ----> mary
45 ----> can
150 ----> speak
138 ----> french
4 ----> .
3 ----> <end>


In [7]:
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)// BATCH_SIZE
embedding_dim_source = 256
embedding_dim_target = 100
units = 1024
vocab_inp_size = len(inp_lang.word_index) + 1
vocab_tar_size = len(targ_lang.word_index) + 1

In [8]:
# transform to torch tensor
tensor_x = torch.Tensor(input_tensor_train).long() 
tensor_y = torch.Tensor(target_tensor_train).long()
# create your datset
my_dataset = data.TensorDataset(tensor_x,tensor_y) 
# create your dataloader
my_dataloader = data.DataLoader(my_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        drop_last=True,
                        num_workers=4)   

In [9]:
example_input_batch, example_target_batch = next(iter(my_dataloader))

In [10]:
example_input_batch.shape

torch.Size([64, 42])

In [26]:
encoder = Encoder(vocab_inp_size, embedding_dim_source, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
encoder_output, encoder_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(encoder_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(encoder_hidden.shape))

Encoder output shape: (batch size, sequence length, units) torch.Size([64, 42, 1024])
Encoder Hidden state shape: (batch size, units) torch.Size([1, 64, 1024])


In [72]:
attention_layer = LuongAttentionDot()

attention_result, attention_weights = attention_layer(encoder_hidden, encoder_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) torch.Size([64, 1024])
Attention weights shape: (batch_size, sequence_length, 1) torch.Size([64, 42, 1])


In [79]:
attention_layer = LuongAttentionGeneral(1024)

attention_result, attention_weights = attention_layer(encoder_hidden, encoder_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) torch.Size([64, 1024])
Attention weights shape: (batch_size, sequence_length, 1) torch.Size([64, 42, 1])


In [101]:
attention_layer = LuongAttentionConcat(10, 1024)

attention_result, attention_weights = attention_layer(encoder_hidden, encoder_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) torch.Size([64, 1024])
Attention weights shape: (batch_size, sequence_length, 1) torch.Size([64, 42, 1])


In [71]:
attention_layer = BahdanauAttention(10, 1024)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

scccccooo torch.Size([64, 42, 1])
Attention result shape: (batch size, units) torch.Size([64, 1024])
Attention weights shape: (batch_size, sequence_length, 1) torch.Size([64, 42, 1])


In [13]:
decoder = Decoder(vocab_tar_size, embedding_dim_target, units, BATCH_SIZE, 1024)

sample_decoder_output, _, _ = decoder(torch.randint(1, 20, (BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) torch.Size([64, 13853])


In [14]:
encoder = Encoder(vocab_inp_size, embedding_dim_source, units, BATCH_SIZE).to(device)
decoder = Decoder(vocab_tar_size, embedding_dim_target, units, BATCH_SIZE, 1024).to(device)

encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())

criterion = nn.NLLLoss()

In [15]:
# Train the model
EPOCHS = 1

for epoch in range(EPOCHS):
    start = time.time()

    total_loss = 0

    for (batch, (inp, targ)) in enumerate(my_dataloader):
        inp, targ = inp.to(device), targ.to(device)
        batch_loss = train_step(inp, targ, encoder, decoder,
                                encoder_optimizer, decoder_optimizer,
                                criterion, device, BATCH_SIZE, targ_lang)
        
        total_loss += batch_loss

        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, batch_loss))
            
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        
        pass

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


Epoch 1 Batch 0 Loss 9.2977
Epoch 1 Batch 100 Loss 1.1881
Epoch 1 Batch 200 Loss 1.4164
Epoch 1 Batch 300 Loss 1.4098
Epoch 1 Batch 400 Loss 1.0159
Epoch 1 Batch 500 Loss 0.9792
Epoch 1 Batch 600 Loss 1.1661
Epoch 1 Batch 700 Loss 0.8869
Epoch 1 Batch 800 Loss 0.8086
Epoch 1 Batch 900 Loss 1.1950
Epoch 1 Batch 1000 Loss 1.2240
Epoch 1 Batch 1100 Loss 0.7379
Epoch 1 Batch 1200 Loss 1.0410
Epoch 1 Batch 1300 Loss 1.0878
Epoch 1 Batch 1400 Loss 1.0608
Epoch 1 Batch 1500 Loss 0.5883
Epoch 1 Batch 1600 Loss 0.8162
Epoch 1 Loss 1.0447
Time taken for 1 epoch 1080.699461221695 sec



In [ ]:
translate(u'trata de averiguarlo .', max_length_targ, max_length_inp, encoder,
          decoder, inp_lang, targ_lang, device, beam_width=10, alpha=0.1)

In [ ]:
translate(u'trata de averiguarlo .', max_length_targ, max_length_inp, encoder,
          decoder, inp_lang, targ_lang, device, beam_width=100, alpha=0.1)

In [ ]:
translate(u'trata de averiguarlo .', max_length_targ, max_length_inp, encoder,
          decoder, inp_lang, targ_lang, device, beam_search=False)

In [ ]:
translate(u'esta es mi vida .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device)

In [ ]:
translate(u'esta es mi vida .', max_length_targ, max_length_inp, encoder,
          decoder, inp_lang, targ_lang, device, beam_width=10, alpha=0.2)

In [ ]:
translate(u'hace mucho frio aqui .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device, beam_search=False)

In [ ]:

translate(u'hace mucho frio aqui .', max_length_targ, max_length_inp, encoder, decoder, inp_lang, targ_lang, device, beam_width=5, alpha=0.2)

# FIN